In [1]:
# stdlib
import os

# syft absolute
import syft as sy
from syft.client.syncing import compare_states
from syft.client.syncing import resolve_single

In [2]:
node_low = sy.orchestra.launch(
    name="reddit_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="reddit_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=777ee6be94024bea8294485360cf7fd3 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=6ce252632e8045a6a1ff9c6c90edc663 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <reddit_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <reddit_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

SyftSuccess: User 'John Doe' successfully registered! To see users, run `[your_client].users`

In [5]:
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <reddit_l: Low side Domain> as <newuser@openmined.org>


# Setup twin api

In [6]:
BQ_PRIVATE_KEY = os.environ["BQ_PRIVATE_KEY"].replace("\\n", "\n")

In [7]:
bq_credentials = {
    "type": "service_account",
    "project_id": "reddit-testing-415005",
    "private_key_id": "0b46406c4b049257f4c6f8f2473a2a05bf3e5321",
    "private_key": BQ_PRIVATE_KEY,
    "client_email": "rdt-testing-sv-acc@reddit-testing-415005.iam.gserviceaccount.com",
    "client_id": "118222186980956490094",
    "auth_uri": "https://accounts.google.com/o/oauth2/auth",
    "token_uri": "https://oauth2.googleapis.com/token",
    "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
    "client_x509_cert_url": "https://www.googleapis.com/robot/v1/metadata/x509/rdt-testing-sv-acc%40reddit-testing-415005.iam.gserviceaccount.com",
    "universe_domain": "googleapis.com",
}

In [8]:
# Mock Behavior


@sy.mock_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def mock_query_function(
    context,  # Variable used to track user session, user information, user activities and settings
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )

    # Replacing private values to mocked ones.
    result = rows.to_dataframe()
    result["int64_field_0"] = 0
    result["id"] = "Private"
    result["name"] = "Private"
    result["subscribers_count"] = 0
    result["permalink"] = "Private"
    result["nsfw"] = "NaN"
    result["spam"] = False
    return result

In [9]:
# Private Behavior


@sy.private_api_endpoint(
    # settings={}
    settings=bq_credentials,
)
def private_query_function(
    context,
    sql_query: str,
) -> str:
    # third party
    from google.cloud import bigquery
    from google.oauth2 import service_account

    print("test")
    credentials = service_account.Credentials.from_service_account_info(
        context.settings
    )
    scoped_credentials = credentials.with_scopes(
        ["https://www.googleapis.com/auth/cloud-platform"]
    )

    client = bigquery.Client(
        credentials=scoped_credentials,
        location="us-west1",
    )
    print(client)

    rows = client.query_and_wait(
        sql_query,
        project=context.settings["project_id"],
    )
    print(rows)

    return rows.to_dataframe()

In [10]:
new_endpoint = sy.TwinAPIEndpoint(
    path="reddit.query",
    private_function=private_query_function,
    mock_function=mock_query_function,
    description="Lorem ipsum dolor sit amet lorem adipiscing elit …",
)
client_high.api.services.api.add(endpoint=new_endpoint)

SyftSuccess: Endpoint successfully created.

In [11]:
client_high.refresh()

In [12]:
client_high.api.services.reddit.query.private(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

test


,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


In [13]:
twin_api_obj = client_high.api.services.api.api_endpoints()[0]

## Sync twin api

In [14]:
# CustomEndpointActionObject

In [15]:
state = client_high.get_sync_state()

In [16]:
state

syft.service.sync.sync_state.SyncState

In [17]:
twin_api_obj = (
    node_high.python_node.service_path_map["apiservice"]
    .stash.get_all(client_high.credentials)
    .ok()[0]
)
twin_api_obj

```python
class TwinAPIEndpoint:
  id: str = b84937a9ec3e461ab9a83349baf51942

```

In [18]:
custom_actions = list(
    node_high.python_node.service_path_map["actionservice"].store.data.values()
)
custom_action = custom_actions[0]
custom_action

```python
Pointer
```
```python
class CustomEndpointActionObject:
  id: str = 8114602ea14047599f814ab1edac8e75

```

In [19]:
# isinstance(custom_action, CustomEndpointActionObject)

In [20]:
node_high.python_node.get_service("actionservice")

What we need to write

In [21]:
# node_low.python_node.service_path_map["apiservice"].stash.upsert(
#     credentials=client_low.credentials, endpoint=twin_api_obj
# )

In [22]:
# client_low.api.services.action.set(custom_action)

In [23]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()
high_state

syft.service.sync.sync_state.SyncState

In [24]:
diff_state = compare_states(high_state, low_state)
widget = resolve_single(diff_state[0])

Decision: Syncing 1 objects


In [25]:
widget

In [25]:
widget.click_sync()

Decision: Syncing 1 objects


SyftSuccess: Synced 1 items

In [26]:
client_low.refresh()
low_state = client_low.get_sync_state()
low_state

syft.service.sync.sync_state.SyncState

In [27]:
twin_api_obj = (
    node_low.python_node.service_path_map["apiservice"]
    .stash.get_all(client_high.credentials)
    .ok()
)
twin_api_obj

[]

In [28]:
node_low.python_node.get_service("actionservice").store.storage_permissions

{}

# Create request

## Use mock endpoint

In [29]:
client_low.refresh()
client_low_ds.refresh()

In [30]:
# client_low.api.services.api.api_endpoints()

In [31]:
# client_low.api.services.reddit.query.mock(sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100")

In [32]:
client_low_ds.api.services.reddit.query(
    sql_query="SELECT *  FROM test_1gb.subreddits LIMIT 100"
)

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,0,Private,Private,0,Private,NaN,False
1,0,Private,Private,0,Private,NaN,False
2,0,Private,Private,0,Private,NaN,False
3,0,Private,Private,0,Private,NaN,False
4,0,Private,Private,0,Private,NaN,False
...,...,...,...,...,...,...,...
95,0,Private,Private,0,Private,NaN,False
96,0,Private,Private,0,Private,NaN,False
97,0,Private,Private,0,Private,NaN,False
98,0,Private,Private,0,Private,NaN,False


# Define code, project, request

In [33]:
@sy.syft_function_single_use(
    reddit_query=client_low_ds.api.services.reddit.query,
)
def my_research_pipeline(reddit_query):
    sql_query = "SELECT *  FROM test_1gb.subreddits LIMIT 100"
    return reddit_query(sql_query=sql_query)

SyftSuccess: Syft function 'my_research_pipeline' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [34]:
new_project = sy.Project(
    name="Reddit Research Studies",
    description="Hi, I want to get information about your data.",
    members=[client_low_ds],
)

new_project.create_code_request(my_research_pipeline, client_low_ds)

SyftSuccess: Code request for 'my_research_pipeline' successfully added to 'Reddit Research Studies' Project. To see code requests by a client, run `[your_client].code`

In [35]:
# low_request = client_low.requests[-1]

In [36]:
# low_request

In [37]:
# low_code = low_request.code

In [38]:
# low_code

## Sync code request to high side

In [39]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [40]:
low_state

syft.service.sync.sync_state.SyncState

In [41]:
# syft absolute

In [42]:
diff_state = compare_states(low_state, high_state)

In [43]:
diff_state

syft.service.sync.diff_state.NodeDiff

### Sync UserCode

In [44]:
obj_diff_batch = diff_state[0]

In [45]:
widget = resolve_single(obj_diff_batch)

In [46]:
widget

In [47]:
widget.click_sync()

Decision: Syncing 2 objects


SyftSuccess: Synced 2 items

### Sync Request

In [48]:
obj_diff_batch_request = diff_state[1]

In [49]:
widget = resolve_single(obj_diff_batch_request)

In [50]:
widget

In [51]:
widget.click_sync()

Decision: Syncing 1 objects


SyftSuccess: Synced 1 items

# High side: Run and sync back

## Run on high side

In [52]:
client_high.refresh()

In [53]:
client_high.requests

In [54]:
request = client_high.requests[0]

In [55]:
# request.code

In [56]:
res = client_high.code.my_research_pipeline(
    reddit_query=client_high.api.services.reddit.query
)

test


In [57]:
res

,int64_field_0,id,name,subscribers_count,permalink,nsfw,spam
0,4,t5_via1x,/r/mylittlepony,4323081,/r//r/mylittlepony,NaN,False
1,5,t5_cv9gn,/r/polyamory,2425929,/r//r/polyamory,NaN,False
2,10,t5_8p2tq,/r/Catholicism,4062607,/r//r/Catholicism,NaN,False
3,16,t5_8fcro,/r/cordcutters,7543226,/r//r/cordcutters,NaN,False
4,17,t5_td5of,/r/stevenuniverse,2692168,/r//r/stevenuniverse,NaN,False
...,...,...,...,...,...,...,...
95,305,t5_jgydw,/r/cannabis,7703201,/r//r/cannabis,NaN,False
96,311,t5_3mfau,/r/marvelmemes,4288492,/r//r/marvelmemes,NaN,False
97,317,t5_ub3c8,/r/ghibli,6029127,/r//r/ghibli,NaN,False
98,319,t5_fbgo3,/r/birdsarentreal,3416317,/r//r/birdsarentreal,NaN,False


In [58]:
request.accept_by_depositing_result(res)

Approving request for domain reddit_h
Creating job for existing user code
Approving request for domain reddit_h
ADDING PERMISSION [READ: 17bda187733b48f0b47625df9cfbd96c as c6a4ab9e8b79e08c6cab57efa445e0cffa3bf6dba16c02fb62bc84dc79ceb656] 17bda187733b48f0b47625df9cfbd96c


UnboundLocalError: cannot access local variable 'job_info' where it is not associated with a value

## Sync back to low side

In [ ]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [ ]:
high_state

In [ ]:
diff_state_2 = compare_states(high_state, low_state)

In [ ]:
diff_state_2

In [ ]:
code_batch = diff_state_2[0]
# code_batch

In [ ]:
widget = resolve_single(code_batch)
widget

In [ ]:
res = widget.click_sync()

In [ ]:
request_batch = diff_state_2[1]

In [ ]:
request_batch

In [ ]:
widget = resolve_single(request_batch)
widget

In [ ]:
# res = widget.click_share_all_private_data()

In [ ]:
res = widget.click_sync()

In [ ]:
res

In [ ]:
job_batch = diff_state_2[2]

In [ ]:
# job_batch

In [ ]:
widget = resolve_single(job_batch)
widget

In [ ]:
widget.click_share_all_private_data()
widget.click_sync()

In [ ]:
# client_low.get_sync_state()

In [ ]:
# %debug

# Run on low side

In [ ]:
client_low_ds.refresh()
client_low_ds.code.my_research_pipeline(
    reddit_query=client_low_ds.api.services.reddit.query
)